In [28]:
## importing necessary libraries
import tensorflow as tf
import pandas as pd
import tensorflow_text as text
import wandb
import tensorflow_hub as hub
from wandb.keras import WandbCallback

## Initializing W&B run

In [2]:
PROJECT_NAME = "banking_77"
JOB_TYPE = "baseline"
ENTITY = "basha"
SPLIT_DATA = "preprocess"

In [3]:
run = wandb.init(project=PROJECT_NAME,job_type=JOB_TYPE,entity=ENTITY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: basha. Use `wandb login --relogin` to force relogin


## Batching data of W&B Artifact
We are downloading preprocesed data from W&B and batching the dataset using tf.data. This will help to use GPU when try to use the dataset.

In [4]:
split = run.use_artifact(f'{SPLIT_DATA}:latest')
split_dir = split.download()

In [6]:
train_df = pd.read_csv(f'{split_dir}//train_split.csv')
valid_df = pd.read_csv(f'{split_dir}//valid_split.csv')
test_df = pd.read_csv(f'{split_dir}//test_split.csv')

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_df['text'],train_df['label'])).batch(32).cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = tf.data.Dataset.from_tensor_slices((valid_df['text'],valid_df['label'])).batch(32).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((test_df['text'],test_df['label'])).batch(32).cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
text_batch_1, text_label_1 = next(iter(train_ds)) # pick first batch from dataset
text , label = text_batch_1[0],text_label_1[0] # taking first text and label from batch
text , label # prints first text and label

(<tf.Tensor: shape=(), dtype=string, numpy=b'Will you send me a new card in China?'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=9>)

### Logging model configuration in W&B

In [23]:
from ml_collections import config_dict

cfg = config_dict.ConfigDict()
cfg.preprocessor = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
cfg.encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'
cfg.bs = 32
cfg.seed = 42
cfg.arch = 'bert_L-2_H-128_A-2'
cfg.learning_rate = 1e-5

In [24]:
wandb.config.update(cfg.to_dict())

## Model building

In [25]:
def make_model():
    text_input = tf.keras.Input(shape=(),dtype=tf.string,name="input_layer")
    preprocessor = hub.KerasLayer(cfg.preprocessor,name="preprocessor")
    encoder_inputs = preprocessor(text_input)

    encoder = hub.KerasLayer(cfg.encoder,trainable=True,name='BERT_encoder')
    outputs = encoder(encoder_inputs)

    pooled_output = outputs['pooled_output']

    #create Dense layer for classifier
    # net = tf.keras.layers.Dropout(0.1)(pooled_output)
    net = tf.keras.layers.Dense(77,activation='softmax')(pooled_output) #predict 77 classes
    return tf.keras.Model(text_input, net)

###  Creating and compiling a model

In [26]:
model = make_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cfg.learning_rate),
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                         metrics=['accuracy'])

### Training a model

In [29]:
model.fit(train_ds,validation_data=valid_ds,epochs=5,callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/5
282/282 [==============================] - 77s 260ms/step - loss: 4.5127 - accuracy: 0.0149 - val_loss: 4.3682 - val_accuracy: 0.0220 - _timestamp: 1658580047.0000 - _runtime: 2778.0000
Epoch 2/5
282/282 [==============================] - 69s 246ms/step - loss: 4.3332 - accuracy: 0.0312 - val_loss: 4.1932 - val_accuracy: 0.0599 - _timestamp: 1658580116.0000 - _runtime: 2847.0000
Epoch 3/5
282/282 [==============================] - 73s 258ms/step - loss: 4.1778 - accuracy: 0.0514 - val_loss: 3.9731 - val_accuracy: 0.1129 - _timestamp: 1658580189.0000 - _runtime: 2920.0000
Epoch 4/5
282/282 [==============================] - 72s 256ms/step - loss: 3.9721 - accuracy: 0.1088 - val_loss: 3.7191 - val_accuracy: 0.1998 - _timestamp: 1658580261.0000 - _runtime: 2992.0000
Epoch 5/5
282/282 [==============================] - 77s 274ms/step - loss: 3.7723 - accuracy: 0.1656 - val_loss: 3.4729 - val_accuracy: 0.2687 - _timestamp: 1658580338.0000 - _runtime: 3069.0000


### Predictions on Test set

In [30]:
preds = model.predict(test_ds)

97/97 [==============================] - 7s 65ms/step


In [31]:
preds = tf.math.argmax(preds,1)

In [37]:
tf.math.argmax(model.predict(tf.constant(['How do I locate my card?'])),1)

1/1 [==============================] - 0s 20ms/step


<tf.Tensor: shape=(1,), dtype=int64, numpy=array([39], dtype=int64)>

In [33]:
test_df['predicted'] = preds

### Logging Predictions table

In [40]:
run.log({'predictions_table': wandb.Table(dataframe=test_df)})

### Evaluating train set

In [41]:
model.evaluate(train_ds) # train set predicitions

282/282 [==============================] - 20s 72ms/step - loss: 3.4553 - accuracy: 0.2728


[3.4553279876708984, 0.27282825112342834]

### Evaluating validation set

In [42]:
model.evaluate(valid_ds) # valid set predicitions

32/32 [==============================] - 2s 71ms/step - loss: 3.4729 - accuracy: 0.2687


[3.4728615283966064, 0.2687312662601471]

### Shutting down W&B run

In [43]:
run.finish()

accuracy,▁▂▃▅█
epoch,▁▃▅▆█
loss,█▆▅▃▁
val_accuracy,▁▂▄▆█
val_loss,█▇▅▃▁
GFLOPS,0.00094
accuracy,0.16563
best_epoch,4
best_val_loss,3.47286
epoch,4
loss,3.77234
